In [11]:
def print_conversion_header(self, file_name, current, total):
    header_line = "=" * 100 + "\n"
    print(f"{header_line *10}{'=' * 29} Processing {file_name} - {current}/{total} {'=' * 29}\n{header_line *10}")


print_conversion_header("x","x","x","x" )

============================= Processing x - x/x =============================

